In [1]:
from transformers import (
    SeamlessM4TModel,
    AutoProcessor
)
import torch
import os
from pydub import AudioSegment
import numpy as np
import io
import soundfile as sf
import librosa
import torchaudio
import gradio as gr

f:\test\chatbot\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
f:\test\chatbot\env\Lib\site-packages\transformers\deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
f:\test\chatbot\env\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [6]:
# URL save model and related layers
save_url = "./seamless_m4t_model"
model_name= "facebook/hf-seamless-m4t-medium"

In [7]:
# Load the model and related layers
try:
  processor = AutoProcessor.from_pretrained(model_name)
  model = SeamlessM4TModel.from_pretrained(model_name)
except OSError as e:
  print(f"Error when load model and related layer: {e}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Save the model and related layers to a local file
model.save_pretrained(save_url)
processor.save_pretrained(save_url)

print(f"Mô hình và các lớp liên quan đã được lưu vào {save_url}")

Mô hình và các lớp liên quan đã được lưu vào ./seamless_m4t_model


In [9]:
# Tải âm thanh từ file và chuyển đổi
input_file = "./data/audio/clean_spk3_00032_pred.wav"
arr, org_sr = torchaudio.load(input_file)

AUDIO_SAMPLE_RATE = 16000.0
new_arr = torchaudio.functional.resample(arr, orig_freq=org_sr, new_freq=AUDIO_SAMPLE_RATE)

MAX_INPUT_AUDIO_LENGTH = 60  # in seconds
max_length = int(MAX_INPUT_AUDIO_LENGTH * AUDIO_SAMPLE_RATE)
if new_arr.shape[1] > max_length:
    new_arr = new_arr[:, :max_length]
    gr.Warning(f"Input audio is too long. Only the first {MAX_INPUT_AUDIO_LENGTH} seconds is used.")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_data = processor(audios = new_arr, sampling_rate=AUDIO_SAMPLE_RATE, return_tensors="pt").to(device)

In [11]:
tokens_ids = model.generate(
  **input_data, 
  generate_speech=False, 
  tgt_lang="eng", 
  num_beams=5, 
  do_sample=True
)[0].cpu().squeeze().detach().tolist()

text_out = processor.decode(tokens_ids, skip_special_tokens=True)

print("trans", text_out)


trans Have you ever considered writing a column of advice?


In [12]:
output = model.generate(
  **input_data, 
  return_intermediate_token_ids=True, 
  tgt_lang="eng", 
  num_beams=5, 
  do_sample=True, 
  spkr_id=10
)
        
waveform = output.waveform.cpu().squeeze().detach().numpy()
tokens_ids = output.sequences.cpu().squeeze().detach().tolist()

text_out = processor.decode(tokens_ids, skip_special_tokens=True)

print("audio", (AUDIO_SAMPLE_RATE, waveform))

audio (16000.0, array([-9.2901588e-05, -1.2566843e-04, -8.7207532e-05, ...,
        2.3517360e-04,  2.1274990e-04,  2.1224630e-04], dtype=float32))
